In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_scores = pd.read_csv('my_data_scores.csv', index_col=0)
df_scores

,user_id,statement_num,my_pd_bucket,x_bucket,my_pd,x_score
0,127336,19,2.0,2.0,0.074753,0.469410
1,35907,16,1.0,0.0,0.058043,0.157215
2,45228,21,1.0,2.0,0.059475,0.469583
3,39035,23,0.0,1.0,0.033255,0.210896
4,130780,19,1.0,1.0,0.046993,0.269492
...,...,...,...,...,...,...
2538699,64006,19,1.0,1.0,0.066447,0.188233
2538700,138666,21,0.0,1.0,0.024477,0.221981
2538701,120568,1,1.0,0.0,0.045209,0.104274
2538702,191861,6,1.0,0.0,0.056931,0.172479


In [4]:
df_scores = df_scores.sort_values(['my_pd'], ignore_index=True)
df_scores['my_pd_bucket'] = np.minimum(df_scores.index // (df_scores.shape[0]//3),2)
print(df_scores.groupby(['my_pd_bucket'])['my_pd'].mean())

df_scores = df_scores.sort_values(['x_score'], ignore_index=True)
df_scores['x_bucket'] = np.minimum(df_scores.index // (df_scores.shape[0]//3),2)
print(df_scores.groupby(['x_bucket'])['x_score'].mean())

df_scores

my_pd_bucket
0    0.025896
1    0.050567
2    0.103601
Name: my_pd, dtype: float64
x_bucket
0    0.132996
1    0.220232
2    0.378351
Name: x_score, dtype: float64


,user_id,statement_num,my_pd_bucket,x_bucket,my_pd,x_score
0,47228,10,0,0,0.024272,0.039711
1,47228,20,0,0,0.024272,0.039711
2,47228,14,0,0,0.024272,0.039711
3,47228,19,0,0,0.024272,0.039711
4,47228,9,0,0,0.024272,0.039711
...,...,...,...,...,...,...
2538627,212993,21,0,2,0.032879,0.793186
2538628,212993,18,0,2,0.032879,0.793186
2538629,212993,11,0,2,0.032879,0.793186
2538630,212993,19,0,2,0.032879,0.793186


In [5]:
mean_pd = df_scores.groupby(['my_pd_bucket'])['my_pd'].mean()
mean_x = df_scores.groupby(['x_bucket'])['x_score'].mean()
mean_scores = pd.DataFrame({'pd': mean_pd, 'x': mean_x})
mean_scores

,pd,x
0,0.025896,0.132996
1,0.050567,0.220232
2,0.103601,0.378351


In [6]:
pd_0 = 0.05
mean_scores[mean_scores['pd'] > pd_0].index[0]

1

In [7]:
x_0 = 0.15
mean_scores[mean_scores['x'] > x_0].index[0]

1

In [10]:
def lin_interpolation(rates, bucket_0, bucket_1, feat_name, feat_0):
    result = []
    mean_score = mean_scores[feat_name]
    for i in df.index:
        result.append(rates[bucket_0][i] +
              (feat_0 - mean_score[float(bucket_0)]) * 
              (rates[bucket_1][i] - rates[bucket_0][i]) / 
              (mean_score[float(bucket_1)] - mean_score[float(bucket_0)]))
    return result

# Кривые для первого задания

In [98]:
database = {}
df = pd.read_csv('clo_rate_by_buckets.csv')
database['clo'] = lin_interpolation(df, '0.0', '1.0', 'x', 0.15)

In [99]:
df = pd.read_csv('def_rate_by_buckets.csv')
database['def'] = lin_interpolation(df, '0.0', '1.0', 'pd', 0.05)

In [100]:
df = pd.read_csv('dlq_ratio_by_buckets.csv')
database['dlq'] = lin_interpolation(df, '0.0', '1.0', 'pd', 0.05)

In [102]:
pd.DataFrame(database).to_excel('result.xlsx')

# Кривые при изменении параметров

## Pd

### +1%

In [8]:
pd_0 = 0.06
mean_scores[mean_scores['pd'] > pd_0].index[0]

2

In [11]:
database = {}
df = pd.read_csv('def_rate_by_buckets.csv')
database['def_pd_p1'] = lin_interpolation(df, '1.0', '2.0', 'pd', 0.06)
df = pd.read_csv('dlq_ratio_by_buckets.csv')
database['dlq_pd_p1'] = lin_interpolation(df, '1.0', '2.0', 'pd', 0.06)

### -1%

In [12]:
pd_0 = 0.04
mean_scores[mean_scores['pd'] > pd_0].index[0]

1

In [13]:
df = pd.read_csv('def_rate_by_buckets.csv')
database['def_pd_m1'] = lin_interpolation(df, '0.0', '1.0', 'pd', 0.04)
df = pd.read_csv('dlq_ratio_by_buckets.csv')
database['dlq_pd_m1'] = lin_interpolation(df, '0.0', '1.0', 'pd', 0.04)

## x

### +1%

In [18]:
x_0 = 0.16
mean_scores[mean_scores['x'] > x_0].index[0]

1

In [19]:
df = pd.read_csv('clo_rate_by_buckets.csv')
database['clo_x_p1'] = lin_interpolation(df, '0.0', '1.0', 'x', 0.16)

### -1%

In [20]:
x_0 = 0.14
mean_scores[mean_scores['x'] > x_0].index[0]

1

In [21]:
df = pd.read_csv('clo_rate_by_buckets.csv')
database['clo_x_m1'] = lin_interpolation(df, '0.0', '1.0', 'x', 0.14)

In [22]:
pd.DataFrame(database).to_excel('result_diff.xlsx')